## Key words
### 평균비교, 일원분산분석, 사후검정, Tukey, f_oneway, ols, anova_lm, pariwise_tukeyhsd, statsmodels

# 일원 분산 분석(One Way ANOVA)
- 평균을 분산을 기반으로 해서 분산 분석이라고함
- 수치형 종속변수와 명목형(범주형) 독립변수가 각각 1개 일 때 실시하는 분석
 - `명목형 독립변수`가 1개라서 일원 분산 분석, 2개면 이원
- 종속변수로 나뉘어지는 `그룹이 2개 이상`일때 사용
 - 2개일때는 보통 t검정을 많이씀
 - `3~4개 넘어갈 때` 사용
- 귀무가설을 기각하는 경우 -> 사후 검정(Post-hoc)을 실시

### 가설
- 귀무가설 : 집단 간 평균이 같음
- 대립가설 : 평균이 같지 않은 집단이 한 쌍 이상 존재

# 일원 분산 분석의 사후 검정
- 목적 : 귀무가설이 기각되었을 때 어디서 뭐가 다른지 알기 위함
- 방법 : Tukey's HSD, Duncan's MRT, Scheffe's test
- 실습 : `Tukey's HSD 사용`
- 모든 집단간 독립 2 표본 t-검정을 하는 것과 유사하며, 어떤 집단간 평균이 유의하게 차이나는지 알 수 있음

## 일원 분산 분석 - f_oneway()
- 일원 분반 분석을 수행하는 scipy의 함수
- 입력은 각각의 집단을 pandas의 series로 하는 것을 권장
- 출력은 검정통계량과 p-value 두 개 이며 하나의 객체에 할당할 경우 튜플로 저장됨

In [1]:
import pandas as pd
from scipy.stats import f_oneway
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [20]:
df = pd.read_csv("diamonds.csv")
df.head(2)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31


In [3]:
df["color"].unique()

array(['E', 'I', 'J', 'H', 'F', 'G', 'D'], dtype=object)

In [4]:
stat, p = f_oneway(df.loc[df["color"] == "E", "price"],
                   df.loc[df["color"] == "I", "price"],
                   df.loc[df["color"] == "J", "price"])
print(round(stat, 3))
print(round(p, 3))

621.846
0.0


- p값이 0 .05를 넘지못하므로 귀무가설 기각
- 집단 간 평균은 유의마하게 차이가 난다
- 적어도 한 쌍 이상 평균이 차이가 난다

In [21]:
df.groupby("color")["price"].mean().reset_index() # 직접 평균들을 보고 비교해보기 E, I, J

,color,price
0,D,3169.954096
1,E,3076.752475
2,F,3724.886397
3,G,3999.135671
4,H,4486.669196
5,I,5091.874954
6,J,5323.818020


## 일원 분산 분석 - ols(), anova_lm()
- statsmodels의 일원 분산 분석을 수행하는 함수
- ols() 함수는 모델을 생성하고 적합
 - 나중에 선형회귀에서 쓰임
- anova_lm() 함수는 적합된 모델정보를 기반으로 일원 분산 분석표를 보여줌
- ols() 함수에 수식 입력 시 독립변수에 C() 함수 사용 권장

### season별로 temp가 유의미하게 다른가?
- ols()로 모델을 만들고
- anova_lm()으로 분석표를 보여줌

In [6]:
bike = pd.read_csv("bike.csv")
bike.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40


In [7]:
model = ols(formula = "temp ~ season", data = bike).fit() # 종속변수: temp, 독립변수: season
anova_lm(model) # 이렇게 쓰면 안됨: season의 value값이 숫자라 숫자로 인식해버림
# 회귀분석처럼 되어버림

,df,sum_sq,mean_sq,F,PR(>F)
season,1.0,44221.657301,44221.657301,780.591754,5.879560e-166
Residual,10884.0,616594.417651,56.651453,NaN,NaN


In [8]:
model = ols(formula = "temp ~ C(season)", data = bike).fit() # C()를 사용하면 안에 변수는 명목형(범주형)이라는 말
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(season),3.0,412885.270005,137628.423335,6040.687453,0.0
Residual,10882.0,247930.804947,22.783570,NaN,NaN


- 첫번째 모델은 season을 수치형으로 인식 -> 회귀분석처럼 됨
- 두번째 모델은 c()를 사용하여 명목형으로 인식하게 해준것

- F: 검정통계량 [해석] 6040
- PR: 검정통계량보다 높은 값이 나올 확률 [해석] 0이나 다름없다


---
### 사후검정 - pairwise_tukeyhsd()
- statsmodels의 Tukey 사후검정을 실시하는 함수
- 함수 내에 종속변수와 독립변수를 차례대로 선언
- 결과를 출력하기 위해서 `print() 함수를 반드시 사용`
- reject 변수의 True는 귀무가설(두 집단의 평균 차이가 유의미하게 나지 않음)을 기각하는 것

In [9]:
result = pairwise_tukeyhsd(bike["temp"], bike["season"])
print(result) # 꼭 프린트로 써줘야 결과값이 나옴

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj  lower    upper   reject
-----------------------------------------------------
     1      2   10.293 0.001   9.9598  10.6262   True
     1      3  16.2586 0.001  15.9254  16.5918   True
     1      4   4.1187 0.001   3.7856   4.4519   True
     2      3   5.9656 0.001   5.6339   6.2974   True
     2      4  -6.1742 0.001   -6.506  -5.8425   True
     3      4 -12.1399 0.001 -12.4716 -11.8081   True
-----------------------------------------------------


- meandiff : 평균차이
- p-adj : p값
- reject : p-value :0.05 기준 유의수준 5%로 귀무가설 기각하겠는가? 
 - p-value 유의수준 0.05이하는 0.95는 평균이 같다는 소리고, 0.05는 같지 않다는 소리

## 2. 계절별 기온 평균을 분산분석 시 검정통계량은?
- bike.csv 파일 사용
- statsmodels 라이브러리 활용

In [10]:
df = pd.read_csv("bike.csv")
df

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


In [11]:
df.info()  # 독립변수 season이 문자형인지 정수형인지 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [12]:
model = ols(formula = "temp ~ C(season)", data = df).fit()
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(season),3.0,412885.270005,137628.423335,6040.687453,0.0
Residual,10882.0,247930.804947,22.783570,NaN,NaN


정답

In [13]:
formula = "temp ~ C(season)"
lm = ols(formula, df).fit()
anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
C(season),3.0,412885.270005,137628.423335,6040.687453,0.0
Residual,10882.0,247930.804947,22.783570,NaN,NaN


## 3. 요일별 registered의 평균을 사후검정 했을 때 평균이 유의미하게 차이 나지 않는 조합은 총 몇개 인가?
- bike.csv 파일 사용

In [14]:
df = pd.read_csv("bike.csv")
df

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


In [15]:
df["datetime"] = pd.to_datetime(df["datetime"])

In [16]:
df["wday"] = df["datetime"].dt.weekday

In [17]:
df

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,wday
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336,2
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241,2
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168,2
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129,2


In [18]:
result = pairwise_tukeyhsd(df["registered"], df["wday"])
print(result)

 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
group1 group2 meandiff p-adj   lower    upper   reject
------------------------------------------------------
     0      1   6.1979    0.9  -9.7188  22.1146  False
     0      2    5.343    0.9 -10.5427  21.2287  False
     0      3  12.7424 0.2132  -3.1383   28.623  False
     0      4   6.2956    0.9  -9.6471  22.2384  False
     0      5 -27.5063  0.001 -43.3091 -11.7036   True
     0      6 -36.7583  0.001 -52.5734 -20.9431   True
     1      2  -0.8549    0.9 -16.7716  15.0618  False
     1      3   6.5445 0.8863  -9.3671  22.4561  False
     1      4   0.0977    0.9 -15.8759  16.0713  False
     1      5 -33.7042  0.001 -49.5381 -17.8704   True
     1      6 -42.9562  0.001 -58.8024 -27.1099   True
     2      3   7.3994 0.7916  -8.4813    23.28  False
     2      4   0.9526    0.9 -14.9901  16.8954  False
     2      5 -32.8493  0.001 -48.6521 -17.0466   True
     2      6 -42.1013  0.001 -57.9164 -26.2861   True
     3    